# Set Coding Environment

## Library installations

In [ ]:
# !pip install geemap geopandas

## Module Imports

In [4]:
# import geemap
# import geopandas as gpd
from pprint import pprint
import ee
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/LULC_Experiments_Chahat_Ananjan_Saketh

/content/drive/MyDrive/LULC_Experiments_Chahat_Ananjan_Saketh


## Authenticate to Google Earth Engine

In [5]:
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=TlAfH9tmVpcZTVwv0bQdk8x8_mgkU5L_utYoo87hbb8&tc=9sxB-BsKMOFbK3rxTfWLvJPaKG_ptDGDJfdRHTUdhr8&cc=CwYFtm0jWY3wGOeHYpqq0UfAxhhzimkk3HfKib7pWkk

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkQu6JEIt3zzva5eI2TkCZG02WQmoBJ7QmuAEI7VDumUHbfozesU4o

Successfully saved authorization token.


In [6]:
ee.Initialize()

# FUNCTION DEFINITIONS

In [7]:
def fill_empty_bands(image):
  band_names = image.bandNames()
  zero_img = image.select(0).multiply(0).rename('constant').toDouble()
  zero_img_masked = zero_img.updateMask(zero_img)
  image = ee.Algorithms.If(ee.List(band_names).contains(ee.String('VV')),image, ee.Image(image).addBands(zero_img_masked.select('constant').rename('VV')))
  image = ee.Algorithms.If(ee.List(band_names).contains(ee.String('VH')),image, ee.Image(image).addBands(zero_img_masked.select('constant').rename('VH')))
  return image


def Get_S1_ImageCollections(inputStartDate, inputEndDate, roi_boundary):
  S1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
         .filter(ee.Filter.eq('instrumentMode', 'IW')) \
         .filterDate(inputStartDate, inputEndDate) \
         .filterBounds(roi_boundary)

  S1_processed = S1.map(fill_empty_bands)
  return S1_processed


def GetVV_VH_image_datewise(S1_ic):
  def get_VV_VH_datewise(date):
    zero_img = S1_ic.first().select('VV','VH').multiply(0)
    zero_img_masked = zero_img.updateMask(zero_img)

    subset_ic = S1_ic.select(['VV','VH']).filterDate(ee.Date(date), ee.Date(date).advance(16, 'day'))
    image = ee.Algorithms.If( ee.Number(subset_ic.size()).gt(0), subset_ic.mean().set('system:time_start',ee.Date(date).millis()), zero_img.set('system:time_start',ee.Date(date).millis()))

    return image
  return get_VV_VH_datewise


def Get_S1_16Day_VV_VH_TimeSeries(inputStartDate, inputEndDate, S1_ic):
  startDate = datetime.strptime(inputStartDate,"%Y-%m-%d")
  endDate = datetime.strptime(inputEndDate,"%Y-%m-%d")

  date_list = pd.date_range(start=startDate, end=endDate, freq='16D').tolist()
  date_list = ee.List( [datetime.strftime(curr_date,"%Y-%m-%d") for curr_date in date_list] )

  S1_TS =  ee.ImageCollection.fromImages(date_list.map(GetVV_VH_image_datewise(S1_ic)))
  return S1_TS


def add_timestamp(image):
  timeImage = image.metadata('system:time_start').rename('timestamp')
  timeImageMasked = timeImage.updateMask(image.mask().select(0))
  return image.addBands(timeImageMasked)


def performInterpolation(image):
  image = ee.Image(image)
  beforeImages = ee.List(image.get('before'))
  beforeMosaic = ee.ImageCollection.fromImages(beforeImages).mosaic()
  afterImages = ee.List(image.get('after'))
  afterMosaic = ee.ImageCollection.fromImages(afterImages).mosaic()

  # Interpolation formula
  # y = y1 + (y2-y1)*((t – t1) / (t2 – t1))
  # y = interpolated image
  # y1 = before image
  # y2 = after image
  # t = interpolation timestamp
  # t1 = before image timestamp
  # t2 = after image timestamp

  t1 = beforeMosaic.select('timestamp').rename('t1')
  t2 = afterMosaic.select('timestamp').rename('t2')
  t = image.metadata('system:time_start').rename('t')
  timeImage = ee.Image.cat([t1, t2, t])
  timeRatio = timeImage.expression('(t - t1) / (t2 - t1)', {
                  't': timeImage.select('t'),
                  't1': timeImage.select('t1'),
                  't2': timeImage.select('t2'),
              })

  interpolated = beforeMosaic.add((afterMosaic.subtract(beforeMosaic).multiply(timeRatio)))
  result = image.unmask(interpolated)

  #Saketh
  #For data points on either end of time-series
  #Before or After mosaics may still have gaps (owing to few/no images in the window)
  #Simply fill with after mosaic (for first few data points) and before mosaic (for last few datapoints)
  fill_value = ee.ImageCollection([beforeMosaic, afterMosaic]).mosaic()
  result = result.unmask(fill_value)

  return result.copyProperties(image, ['system:time_start'])


def interpolate_timeseries(S1_TS):
  filtered = S1_TS.map(add_timestamp)

  # Time window in which we are willing to look forward and backward for unmasked pixel in time series
  timeWindow = 120

  # Define a maxDifference filter to find all images within the specified days. Convert days to milliseconds.
  millis = ee.Number(timeWindow).multiply(1000*60*60*24)
  # Filter says that pick only those timestamps which lie between the 2 timestamps not more than millis difference apart
  maxDiffFilter = ee.Filter.maxDifference(
                              difference = millis,
                              leftField = 'system:time_start',
                              rightField = 'system:time_start',
                            )

  # Filter to find all images after a given image. Compare the image's timstamp against other images.
  # Images ahead of target image should have higher timestamp.
  lessEqFilter = ee.Filter.lessThanOrEquals(
                            leftField = 'system:time_start',
                            rightField = 'system:time_start'
                          )

  # Similarly define this filter to find all images before a given image
  greaterEqFilter = ee.Filter.greaterThanOrEquals(
                            leftField = 'system:time_start',
                            rightField = 'system:time_start'
                          )

  # Apply first join to find all images that are after the target image but within the timeWindow
  filter1 = ee.Filter.And( maxDiffFilter, lessEqFilter )
  join1 = ee.Join.saveAll(
                  matchesKey = 'after',
                  ordering = 'system:time_start',
                  ascending = False
          )
  join1Result = join1.apply(
                  primary = filtered,
                  secondary = filtered,
                  condition = filter1
                )

  # Apply first join to find all images that are after the target image but within the timeWindow
  filter2 = ee.Filter.And( maxDiffFilter, greaterEqFilter )
  join2 = ee.Join.saveAll(
                  matchesKey = 'before',
                  ordering = 'system:time_start',
                  ascending = True
          )
  join2Result = join2.apply(
                  primary = join1Result,
                  secondary = join1Result,
                  condition = filter2
                )

  interpolated_S1_TS = ee.ImageCollection(join2Result.map(performInterpolation))

  return interpolated_S1_TS


def get_trained_model(training_data_assetpath):
  training_data = ee.FeatureCollection(training_data_assetpath)

  # training_band_names = ['0_VH', '1_VH', '2_VH', '3_VH', '4_VH', '5_VH', '6_VH', '7_VH', '8_VH', '9_VH', '10_VH', '11_VH', '12_VH', '13_VH', '14_VH', '15_VH', '16_VH', '17_VH', '18_VH', '19_VH', '20_VH', '21_VH', '22_VH']

  # training_band_names = ['0_VV', '1_VV', '2_VV', '3_VV', '4_VV', '5_VV', '6_VV', '7_VV', '8_VV', '9_VV', '10_VV', '11_VV', '12_VV', '13_VV', '14_VV', '15_VV', '16_VV', '17_VV', '18_VV', '19_VV', '20_VV', '21_VV', '22_VV']

  training_band_names = ['0_VV', '1_VV', '2_VV', '3_VV', '4_VV', '5_VV', '6_VV', '7_VV', '8_VV', '9_VV', '10_VV', '11_VV', '12_VV', '13_VV', '14_VV', '15_VV', '16_VV', '17_VV', '18_VV', '19_VV', '20_VV', '21_VV', '22_VV',
                '0_VH', '1_VH', '2_VH', '3_VH', '4_VH', '5_VH', '6_VH', '7_VH', '8_VH', '9_VH', '10_VH', '11_VH', '12_VH', '13_VH', '14_VH', '15_VH', '16_VH', '17_VH', '18_VH', '19_VH', '20_VH', '21_VH', '22_VH']

  trained_model = ee.Classifier.smileRandomForest(numberOfTrees=100, seed=42).setOutputMode('MULTIPROBABILITY').train(
                              features = training_data,
                            classProperty = 'class',
                            inputProperties = training_band_names )

  return trained_model


def Get_SAR_TS_L2_Output(startDate, endDate, roi_boundary, trained_model):
  S1_ic = Get_S1_ImageCollections(startDate, endDate, roi_boundary)

  S1_TS = Get_S1_16Day_VV_VH_TimeSeries(startDate, endDate, S1_ic)
  # pprint(S1_TS.first().bandNames().getInfo())

  interpolated_S1_TS = interpolate_timeseries(S1_TS)

  S1_TS_img = interpolated_S1_TS.toBands()

  S1_VV_TS_img = S1_TS_img.select(['.*_VV'])
  S1_VH_TS_img = S1_TS_img.select(['.*_VH'])

  training_band_names = ['0_VV', '1_VV', '2_VV', '3_VV', '4_VV', '5_VV', '6_VV', '7_VV', '8_VV', '9_VV', '10_VV', '11_VV', '12_VV', '13_VV', '14_VV', '15_VV', '16_VV', '17_VV', '18_VV', '19_VV', '20_VV', '21_VV', '22_VV',
                '0_VH', '1_VH', '2_VH', '3_VH', '4_VH', '5_VH', '6_VH', '7_VH', '8_VH', '9_VH', '10_VH', '11_VH', '12_VH', '13_VH', '14_VH', '15_VH', '16_VH', '17_VH', '18_VH', '19_VH', '20_VH', '21_VH', '22_VH']

  training_img = S1_VV_TS_img.addBands(S1_VH_TS_img).select(training_band_names).clip(roi_boundary.geometry())
  classified_image = training_img.classify(trained_model)

  roi_label_image = classified_image.select(['classification']).arrayArgmax().arrayFlatten([['Predicted_L2_Label']])
  roi_confidence_image = classified_image.select(['classification']).arrayGet(roi_label_image).rename(['confidence_L2'])
  roi_label_image = roi_label_image.add(5).toInt8()

  LULC_image = roi_label_image.addBands(roi_confidence_image)

  return LULC_image


def Get_slope(roi_boundary):
  dem = ee.Image('CGIAR/SRTM90_V4')
  slope = ee.Terrain.slope(dem)
  slope_image = slope.clip(roi_boundary.geometry())
  return slope_image

# L2 LULC CLASSIFICATION

## Take user input on ROI

In [8]:
# example- projects/ee-indiasat/assets/IndiaSat_test_polygons
# example- projects/ee-indiasat/assets/India_Boundary
# example- projects/ee-indiasat/assets/india_district_boundaries
# example- projects/ee-indiasat/assets/India_state_boundaries
# example- projects/ee-indiasat/assets/mandya_jaltol_boundary
# example- projects/ee-ananjan/assets/Wassan
roi_shapefile_path = input("\n Enter the shapefile path containing your Region Of Interest (ROI): \t")

choice = input("\n Do you want to enter name of your ROI to filter within the shapefile?  \n 1. Yes \n 2. No \n")

if choice == '1':
  roi_name = input("\n Enter the name of your ROI : \t")
  filename_prefix = roi_name
else:
  roi_name = ''
  filename_prefix = input('\nEnter the output filename prefix of your prediction maps (areaName): \t')

# Read the shapefile as feature collection
if roi_name == '':
  roi_boundary = ee.FeatureCollection(roi_shapefile_path)
else:
  roi_boundary = ee.FeatureCollection(roi_shapefile_path).filter(ee.Filter.eq('Name',roi_name))

# pprint(roi_boundary.getInfo())


 Enter the shapefile path containing your Region Of Interest (ROI): 	projects/ee-indiasat/assets/india_district_boundaries

 Do you want to enter name of your ROI to filter within the shapefile?  
 1. Yes 
 2. No 
1

 Enter the name of your ROI : 	Baran


In [ ]:
roi_boundary = ee.FeatureCollection('users/mtpictd/agro_eco_regions').filter(ee.Filter.eq('ae_regcode',13))
filename_prefix = 'EcoRegion13'

In [9]:
training_data_assetpath = 'projects/ee-indiasat/assets/Rasterized_Groundtruth/L2_TrainingData_SAR_TimeSeries_1Year'
trained_model = get_trained_model(training_data_assetpath)

slope_img = Get_slope(roi_boundary)

'''
INFERENCE CODE
'''
startDate = '2015-07-01'
endDate = '2022-07-01'

loopStart = startDate
loopEnd = (datetime.strptime(endDate,"%Y-%m-%d")+relativedelta(years=1)).strftime("%Y-%m-%d")

while loopStart != loopEnd:
  currStartDate = datetime.strptime(loopStart,"%Y-%m-%d")
  currEndDate = (currStartDate+relativedelta(years=1)-timedelta(days=1)).strftime("%Y-%m-%d")
  loopStart = (currStartDate+relativedelta(years=1)).strftime("%Y-%m-%d")

  currStartDate = currStartDate.strftime("%Y-%m-%d")

  print("\n EXECUTING L2 LULC PREDICTION FOR ",currStartDate," TO ",currEndDate,"\n")

  curr_filename = filename_prefix + '_' + currStartDate + "_" + currEndDate

  indiasat_ROI_LULC_image = Get_SAR_TS_L2_Output(currStartDate, currEndDate, roi_boundary, trained_model)
  combined_img = indiasat_ROI_LULC_image.addBands(slope_img)

  #check if the slope is >20 deg, re-classify the pixel from cropland to non-cropland
  final_classified_img = combined_img.select(['Predicted_L2_Label']).where(
                                              combined_img.select('Predicted_L2_Label').eq(5)
                                              .And(
                                                    combined_img.select('slope').gte(30)
                                            ),
                                      6
                                  )

  final_classified_img = final_classified_img.addBands(combined_img.select(['confidence_L2']))
  scale = 30
  # final_output_filename = curr_filename+'_Level2_LULCmap_'+str(scale)+'m'
  final_output_filename = curr_filename+'_Level2_LULCmap_'+str(scale)+'m'
  final_output_assetid = 'projects/ee-indiasat/assets/LULC_Deliverables_WithConfidence/Modified/' + final_output_filename

  # Setup the task
  image_export_task = ee.batch.Export.image.toAsset(
    image = final_classified_img.clip(roi_boundary.geometry()),
    description = final_output_filename,
    assetId = final_output_assetid,
    pyramidingPolicy = {'Predicted_L2_Label': 'mode'},
    scale = scale,
    maxPixels = 1e13,
    crs = 'EPSG:4326'
  )

  image_export_task.start()



 EXECUTING L2 LULC PREDICTION FOR  2015-07-01  TO  2016-06-30 


 EXECUTING L2 LULC PREDICTION FOR  2016-07-01  TO  2017-06-30 


 EXECUTING L2 LULC PREDICTION FOR  2017-07-01  TO  2018-06-30 


 EXECUTING L2 LULC PREDICTION FOR  2018-07-01  TO  2019-06-30 


 EXECUTING L2 LULC PREDICTION FOR  2019-07-01  TO  2020-06-30 


 EXECUTING L2 LULC PREDICTION FOR  2020-07-01  TO  2021-06-30 


 EXECUTING L2 LULC PREDICTION FOR  2021-07-01  TO  2022-06-30 


 EXECUTING L2 LULC PREDICTION FOR  2022-07-01  TO  2023-06-30 

